In [16]:
# Imports 📥

# Packages
#-----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Settings
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # to see all columns in DataFrames


# Data 
# -----------------------------------------------------------------------
abc_coorp_df = pd.read_csv("HR RAW DATA.csv", index_col=0)

In [22]:
# DataTransformer class definition ✍️

class DataTransformer:
    def __init__(self, dataframe):
        self.df = dataframe

    def replace_gender_values(self):
        """Replaces the values in the Gender column with 'Male' and 'Female'."""
        self.df['Gender'] = self.df['Gender'].replace({0: 'Male', 1: 'Female'})

    def convert_object_to_float_eliminate_dolar(self, column_name):
        """
        Converts a column with object type values containing $ signs at the end
        and commas as decimal separators to numeric float values.
        """
        if column_name in self.df.columns:
            # Remove dollar signs from the end
            self.df[column_name] = self.df[column_name].str.replace('$', '', regex=False)
            
            # Replace commas with dots for decimal points
            self.df[column_name] = self.df[column_name].str.replace(',', '.', regex=False)
            
            # Convert the cleaned column to float type
            self.df[column_name] = self.df[column_name].astype(float)
        else:
            print(f"Column {column_name} doesn't exist in the DataFrame.")
        

    def convert_role_to_department_normalize_job_role(self):
        # First change de type of data to be capitalize and the same way
        self.df['JobRole'] = self.df['JobRole'].str.title()
        self.df['Department'] = self.df['Department'].str.title()
       # Clean empty spaces
        self.df['JobRole'] = self.df['JobRole'].str.strip()
        self.df['Department'] = self.df['Department'].str.strip()
        
        conversion_dictionary = {
        'Healthcare Representative': 'Research & Development',
        'Sales Executive': 'Sales',
        'Healthcare Representative': 'Research & Development',
        'Laboratory Technician': 'Research & Development',
        'Manufacturing Director': 'Research & Development',
        'Research Scientist': 'Research & Development',
        'Sales Executive': 'Sales',
        'Sales Representative':'Sales',
        'Research Director': 'Research & Development',
        'Human Resources': 'Human Resources',
         }
        # Iterate over the rows of the DataFrame
        for index, row in self.df.iterrows():
            job_role = row['JobRole']
            # Assign the corresponding value to Department using the dictionary
            if job_role in conversion_dictionary:
                self.df.at[index, 'Department'] = conversion_dictionary[job_role]
                print(f"Value '{job_role}' was transformed into '{conversion_dictionary[job_role]}'")

    def change_marital_status(self):
        
        self.df['MaritalStatus'] = self.df['MaritalStatus'].replace({
        "Marreid": "Married",
        "divorced": "Divorced"})
        self.df["MaritalStatus"] = self.df["MaritalStatus"].fillna("Unknown")
        
    def map_column_remote_work(self):
        dicc = {1: "True", 0: "False", "Yes": "True"}

        # Replace the values in the RemoteWork column according to the dictionary
        self.df["RemoteWork"] = self.df["RemoteWork"].replace(dicc)

In [24]:
# Instantiating a class object 🐣

abc_data = DataTransformer(abc_coorp_df)

In [ ]:
# ▶️ convert_object_to_float_eliminate_dolar

abc_data.convert_object_to_float_eliminate_dolar()
print(abc_data)

abc_data.df["DailyRate"].sample(50)
abc_data.df["DailyRate"].unique()
abc_data.df["DailyRate"].value_counts()
abc_coorp_df

In [ ]:
# ▶️ Convert_role_to_department and normalize jobrole letters
abc_data.convert_role_to_department_normalize_job_role()
print(abc_data)

abc_data.df["JobRole"].sample(50)
abc_data.df["JobRole"].unique()
abc_data.df["Department"].value_counts()
abc_data.df[["JobRole","Department"]].sample(50)
abc_coorp_df

In [ ]:
# ▶️ Transform "marital_status" data

abc_data.change_marital_status()
abc_data.df["MaritalStatus"].unique()
abc_data.df["MaritalStatus"].value_counts()
abc_data.df[["MaritalStatus"]].sample(50)
abc_coorp_df



In [6]:
# ▶️ Transform "remote work"
abc_data.map_column_remote_work()
abc_coorp_df["RemoteWork"].unique()

abc_coorp_df["RemoteWork"].value_counts()

RemoteWork
True     640
1        360
0        309
False    305
Name: count, dtype: int64

In [28]:
# ▶️ Eliminate duplicated "employee_number"

duplicate_employeenumbers = abc_coorp_df['employeenumber'].value_counts()

abc_coorp_df['employeenumber'].isnull().sum()

duplicate_employeenumbers

employeenumber
482,0     2
530,0     2
507,0     2
517,0     2
522,0     2
         ..
161,0     1
164,0     1
190,0     1
194,0     1
2040,0    1
Name: count, Length: 1079, dtype: int64

In [12]:
#employeenumber`: Variable categórica nominal, al tratarse de un identificador único para cada empleado (número). 
# Pero estos números están en formato de texto y posiblemente con un formato de decimal que en realidad no es necesario. 
# Hay 534 duplicados en esta columna. Los valores deben ser unicos.  Hablamos con César. debemos analizar los duplicados mejor 
# y tomar decisiones en base a las conclusiones. Por lo que vimos parece que la diferencia entre los duplicados está en Remote Work, 
# puede que sea un error en la carga de datos o que se haya hecho mas de una vez la encuesta. Explorar mas.

duplicated = abc_coorp_df.loc[abc_coorp_df['employeenumber'].duplicated(keep=False), :].sort_values(by='employeenumber')

    
# Identify duplicates
abc_coorp_df.loc[abc_coorp_df['employeenumber'].duplicated(keep=False), :].sort_values(by='employeenumber')

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
987,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,Not Available,2,1,sALES ReprEsentatiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sALES ReprEsentatiVE - Sales,NaN,False
1483,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,88,2,1,sales repResENTaTiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sales repResENTaTiVE - Sales,NaN,0
1484,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,reSEArcH SCiEnTIst,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,Yes
988,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,rEsEaRCH SCIenTiSt,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,1
990,46,No,NaN,"430,0$",NaN,-47,4,Medical,1,"1069,0",47,0,40,3,5,RESeArCH dIrEcTOR,4,NaN,NaN,21445,9,NaN,No,17,"3,0",4,"80,0",2,"23,0",0,"3,0",2,NaN,2,2,NaN,1977,1000000000$,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,28,No,NaN,"760,0$",Sales,-43,4,NaN,1,NaN,43,1,81,3,2,SaLEs ExECUtiVE,2,NaN,NaN,3698,1,Y,NaN,20,"4,0",1,NaN,0,"8,0",2,"3,0",8,NaN,7,5,NaN,1995,1000000000$,SaLEs ExECUtiVE - Sales,NaN,0
1602,26,Yes,NaN,nan$,NaN,5,2,Medical,1,NaN,3,1,88,2,1,reSearCH scientiST,3,Married,"2366,0",20898,1,NaN,NaN,14,"3,0",1,NaN,1,"8,0",2,"3,0",8,NaN,1,7,"2366,0",1997,1000000000$,NaN,NaN,True
1607,31,No,NaN,"616,0$",NaN,12,3,Medical,1,NaN,4,1,41,3,2,HEaltHCarE REPreSENtAtIve,4,NaN,NaN,17369,0,NaN,NaN,11,"3,0",3,NaN,2,"10,0",2,"1,0",9,NaN,8,5,NaN,1992,1000000000$,NaN,NaN,True
1611,39,No,travel_rarely,"903,0$",NaN,-13,5,NaN,1,NaN,13,0,41,4,3,sAlES ExECUTivE,3,Single,NaN,2560,0,NaN,No,18,"3,0",4,NaN,0,"9,0",3,"3,0",8,NaN,0,7,NaN,1984,1000000000$,NaN,NaN,Yes


## Dirty code below ⬇️⬇️

In [ ]:
def change_commas(string):
    """
    Replaces commas with periods in a given string that represents a decimal number
    in international format (with commas as thousand separators and period as decimal separator).

    Note:
        If an error occurs during the replacement process (e.g., if the argument is not a string),
        the function will return np.nan (Not a Number) to indicate an invalid or unavailable value.
    """

    try:
        # Replace commas with periods in the string
        return float(string.replace(",", "."))
    
    except:
        # If an error occurs (e.g., if the argument is not a string),
        # return np.nan (Not a Number) to indicate an invalid or unavailable value.
        return np.nan


In [ ]:
def categorize_age(number):
    """
    Categorizes age into specific groups.

    This function takes an input number representing age and returns a specific category
    based on the age range.
    """
    if number >= 17 and number <= 25:
        return "Young Adults"
    
    elif number >= 26 and number <= 39:
        return "Young Adults"

    elif number >= 40 and number <= 59:
        return "Middle-aged"
    
    else:
        return "Older Adults"

# We have already created the function and verified that it works.
# The next step is to apply it to our entire DataFrame using the `apply()` method.
# This will return a Series, but we haven't stored this result in a variable.
# So, the next thing we'll do is create a new column in the DataFrame with the result of this apply.
df["age_category"] = df["age"].apply(categorize_age)
